# `qpub` for small python projects

## a single python script

In [10]:
import pytest
@pytest.fixture
def single_script(pytester):
    import nbformat, dgaf
    name = "sample_python_project"
    pytester.makefile(".py", **{name: """'a sample project'
    __version__ = '0.0.1'"""})
    project = dgaf.Project(pytester.path)
    yield project

In [11]:
def test_project(single_script):
    import dgaf
    assert single_script.fs.files
    assert single_script.get_name() == "sample_python_project"
    assert "nbval" not in single_script.get_test_requires()
    assert not single_script.get_url()
    assert single_script.get_requires() == ["pandas"]

In [12]:
def test_flit(pytester, single_script):
    # when there is no url we can initialize one in a few ways.
    import dgaf, shutil
    pyproject = single_script/dgaf.PYPROJECT_TOML
    assert not single_script.exists()
    single_script.to_flit()
    assert single_script.exists()
    result = pytester.run(*"flit build".split())
    assert not result.ret

In [1]:
def test_setuptools(pytester, single_script):
    import dgaf
    setup = single_script/dgaf.SETUP_CFG
    assert not setup.exists()
    single_script.to_setuptools()
    assert setup.exists()
    single_script.to_setup_py()
    result = pytester.run(*"python setup.py build --dry-run".split())
    assert not result.ret